In [1]:
import pandas as pd

import os

import numpy as np

import networkx as nx

import matplotlib.pyplot as plt

from IPython.display import clear_output


In [2]:
#importo il grafo completo
first_graph = nx.read_gpickle('data/first_graph.gpickle')

In [17]:
gephi_graph = nx.read_gexf('data/gephi_graph.gexf')

In [3]:
df_youtuber_videos = pd.read_csv('data/get_youtuber_videos_output/youtuber_videos.csv')
df_youtubers = pd.read_csv('data/get_youtubers_output/youtubers.csv')
df_comments = pd.read_csv('data/get_comment_threads_output/comments_thread_sentiment_reduced_5.csv')
df_videos = pd.read_csv('data/get_videos_details/videos_details_computed.csv')

In [4]:
youtuber_video_join=df_youtubers.set_index('channel_id').join(df_youtuber_videos.set_index('channel_id'), on='channel_id',lsuffix='_yt', rsuffix='_ytvd')
youtuber_video_join = youtuber_video_join[youtuber_video_join.columns[[0,2]]]
youtuber_video_join = youtuber_video_join.set_index('video_id').join(df_videos.set_index('video_id'),on='video_id',lsuffix='_yt', rsuffix='_ytvd')

In [5]:
topics_set =set()
brands_set = set()
youtubers_set = set()

In [6]:
#per set
for idx, n in enumerate(first_graph.nodes):
    print('{}/{}'.format(idx,len(first_graph.nodes)))
    clear_output(wait=True)
    topics = []
    brands= []
    youtubers= []
    user_name = n
    video_idx = df_comments.set_index('author_display_name').loc[[user_name],['video_id']]['video_id'][0]
    row = youtuber_video_join.loc[[video_idx],:]
    [topics.append(item.strip()[1:-1]) for item in row['topics'][0][1:-1].split(',') if item.strip()[1:-1]!= ""] if type(row['topics'][0]) is str else []
    [brands.append(item.strip()[1:-1]) for item in row['brands'][0][1:-1].split(',') if item.strip()[1:-1]!= ""] if type(row['brands'][0]) is str else []
    [youtubers.append(item.strip()[1:-1]) for item in row['youtubers'][0][1:-1].split(',') if item.strip()[1:-1]!= ""] if type(row['youtubers'][0]) is str else []
    if row['display_name'][0].strip()!= "":        
        youtubers.append(row['display_name'][0])
    [topics_set.add(item) for item in topics]
    [brands_set.add(item) for item in brands]
    [youtubers_set.add(item) for item in youtubers]
    #first_graph.nodes[n]['topics'] = topics
    #first_graph.nodes[n]['brands'] = brands
    #first_graph.nodes[n]['youtubers'] = youtubers
    

10576/10577


In [7]:
topics_complete = topics_set.union(brands_set).union(youtubers_set)
len(topics_complete)

238

In [31]:
for n in gephi_graph.nodes:
    for t in topics_complete:
        gephi_graph.nodes[n][t] = 0

In [8]:
for n in first_graph.nodes:
    for t in topics_complete:
        first_graph.nodes[n][t] = 0

In [9]:
#per first_graph
for idx, n in enumerate(first_graph.nodes):
    print('{}/{}'.format(idx,len(first_graph.nodes)))
    clear_output(wait=True)
    topics = []
    brands= []
    youtubers= []
    user_name = n
    video_idx = df_comments.set_index('author_display_name').loc[[user_name],['video_id']]['video_id'][0]
    row = youtuber_video_join.loc[[video_idx],:]
    if type(row['topics'][0]) is str:
        for item in row['topics'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                first_graph.nodes[n][item.strip()[1:-1]] = 1 
    if type(row['brands'][0]) is str:
        for item in row['brands'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                first_graph.nodes[n][item.strip()[1:-1]] = 1
    if type(row['youtubers'][0]) is str:
        for item in row['youtubers'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                first_graph.nodes[n][item.strip()[1:-1]] = 1 
    if row['display_name'][0].strip()!= "":  
        first_graph.nodes[n][row['display_name'][0]] = 1


10576/10577


In [10]:
nx.write_gpickle(first_graph,'data/first_graph_node_attributes_coded.gpickle')

In [34]:
#per gephi_graph
for idx, n in enumerate(gephi_graph.nodes):
    print('{}/{}'.format(idx,len(gephi_graph.nodes)))
    clear_output(wait=True)
    topics = []
    brands= []
    youtubers= []
    user_name = n
    video_idx = df_comments.set_index('author_display_name').loc[[user_name],['video_id']]['video_id'][0]
    row = youtuber_video_join.loc[[video_idx],:]
    if type(row['topics'][0]) is str:
        for item in row['topics'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                gephi_graph.nodes[n][item.strip()[1:-1]] = 1 
    if type(row['brands'][0]) is str:
        for item in row['brands'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                gephi_graph.nodes[n][item.strip()[1:-1]] = 1
    if type(row['youtubers'][0]) is str:
        for item in row['youtubers'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                gephi_graph.nodes[n][item.strip()[1:-1]] = 1 
    if row['display_name'][0].strip()!= "":  
        gephi_graph.nodes[n][row['display_name'][0]] = 1


10576/10577


for n in gephi_graph.nodes:
    del gephi_graph.nodes[n]['topics']
    del gephi_graph.nodes[n]['brands']
    del gephi_graph.nodes[n]['youtubers']

In [9]:
nx.set_node_attributes(first_graph, topics, 'topics')
nx.set_node_attributes(first_graph, brands, 'brands')
nx.set_node_attributes(first_graph, youtubers, 'youtubers')

In [35]:
nx.write_gpickle(gephi_graph,'data/gephi_graph_node_attributes.gpickle')

In [20]:
nx.write_gpickle(first_graph,'data/first_graph_node_attributes.gpickle')

In [36]:
nx.write_gexf(gephi_graph,'data/gephi_graph_node_attributes.gexf')